In [1]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
from numpy import ravel
import pylab as pl
from skimage import transform
import h5py
from sklearn import cross_validation
import uuid
import random
from skimage import io, exposure, img_as_uint, img_as_float
from numpy import (array, dot, arccos)
from numpy.linalg import norm

In [3]:
#求两个点对距离
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [4]:
#将图像直方图均衡化
def image_histogram_equalization(image, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, normed=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = 255 * cdf / cdf[-1] # normalize

    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)

    return image_equalized.reshape(image.shape), cdf

In [82]:
#读入training及test数据，第一行为header
df = pd.read_csv('data/training.csv',header=0)        #(7049, 31)
dfp = pd.read_csv('data/test.csv',header=0)           #(1783, 2)

数据：共7049张图片，96×96
标签：15组（x,y）
df前面30维为15组(x,y)坐标，第31维为图像，大小为32691=

In [66]:
df = df.dropna()  #删除含有nan的所有行  (2140,30)
y = df.drop(['Image'], axis=1)  #(2140,30) 删除图像所在列，留下标签

In [67]:
y = y.values 
y = y.astype(np.float32) 
y = y.reshape((-1,30))      #将标签转换为(2140,30)的二位数组

In [68]:
y_features = np.zeros([len(y),15], np.float32)     #(2140,15)

In [69]:
for k in xrange(len(y)):
 mouth_right_x = y[k,24]
 mouth_right_y = y[k,25]
 mouth_left_x =  y[k,22]
 mouth_left_y = y[k,23]
 mouth_top_x =  y[k,26]
 mouth_top_y =  y[k,27]
 mouth_bottom_x = y[k,28]
 mouth_bottom_y = y[k,29]
 
 left_eye_brow_left_x = y[k,14]
 left_eye_brow_left_y = y[k,15]
 left_eye_brow_right_x = y[k,12]
 left_eye_brow_right_y = y[k,13]
 
 right_eye_brow_left_x = y[k,16]
 right_eye_brow_left_y = y[k,17]
 right_eye_brow_right_x = y[k,18]
 right_eye_brow_right_y = y[k,19]
 
 left_eye_x = y[k,2]
 left_eye_y = y[k,3]
 right_eye_x = y[k,0]
 right_eye_y = y[k,1]

 nose_center_x = y[k,20]
 nose_center_y = y[k,21]

 left_eye_right_corner_x = y[k,4]
 left_eye_right_corner_y = y[k,5]
 left_eye_left_corner_x =  y[k,6]
 left_eye_left_corner_y =  y[k,7]

 right_eye_right_corner_x = y[k,10]
 right_eye_right_corner_y = y[k,11]
 right_eye_left_corner_x =  y[k,8]
 right_eye_left_corner_y =  y[k,9]

 
 nose_center = np.array([nose_center_x,nose_center_y],np.float32)
 
 left_eye = np.array([left_eye_x, left_eye_y], np.float32)
 right_eye = np.array([right_eye_x, right_eye_y], np.float32)
 
 mouth_left = np.array([mouth_left_x, mouth_left_y], np.float32)
 mouth_right = np.array([mouth_right_x, mouth_right_y], np.float32)
 mouth_top  = np.array([mouth_top_x,mouth_top_y],np.float32)
 mouth_bottom = np.array([mouth_bottom_x, mouth_bottom_y],np.float32)

 left_eye_right_corner = np.array([left_eye_right_corner_x,left_eye_right_corner_y])
 left_eye_left_corner = np.array([left_eye_left_corner_x,left_eye_left_corner_y])
 right_eye_right_corner = np.array([right_eye_right_corner_x,right_eye_right_corner_y]) 
 right_eye_left_corner = np.array([right_eye_left_corner_x,right_eye_left_corner_y])

 left_eye_brow_left = np.array([left_eye_brow_left_x,left_eye_brow_left_y],np.float32)
 left_eye_brow_right = np.array([left_eye_brow_right_x,left_eye_brow_right_y],np.float32)
 right_eye_brow_left = np.array([right_eye_brow_left_x,right_eye_brow_left_y],np.float32)
 right_eye_brow_right = np.array([right_eye_brow_right_x,right_eye_brow_right_y],np.float32)


 y_features[k,0] = dist(left_eye,right_eye)
 y_features[k,1] = dist(nose_center,left_eye)
 y_features[k,2] = dist(nose_center,right_eye)
 y_features[k,3] = dist(nose_center,mouth_left)
 y_features[k,4] = dist(nose_center,mouth_right) 
 y_features[k,5] = dist(nose_center, mouth_top)
 y_features[k,6] = dist(left_eye,left_eye_left_corner)
 y_features[k,7] = dist(left_eye, left_eye_right_corner)
 y_features[k,8] = dist(right_eye,right_eye_left_corner)
 y_features[k,9] = dist(right_eye,right_eye_right_corner)
 y_features[k,10] = dist(left_eye_left_corner, left_eye_brow_left)
 y_features[k,11] = dist(left_eye_right_corner, left_eye_brow_right)
 y_features[k,12] = dist(right_eye_left_corner, right_eye_brow_left)
 y_features[k,13] = dist(right_eye_right_corner, right_eye_brow_right)
 y_features[k,14] = dist(mouth_top, mouth_bottom)


In [70]:
print 'Distance Features', y_features.shape
print y_features.reshape(-1,15)

Distance Features (2140, 15)
[[ 35.89942932  25.05351257  28.16822052 ...,   9.23731709  10.5022707
   11.55174732]
 [ 34.41730499  28.75241852  26.23318672 ...,  10.59743214   9.74182034
   15.26218891]
 [ 34.15326309  24.98778343  25.55961037 ...,   9.34853077   9.3484621
    8.4675827 ]
 ..., 
 [ 39.54855728  35.4339447   28.90300179 ...,   9.86591244  11.3681488
   10.69087887]
 [ 38.51984024  33.00073242  30.7424221  ...,   7.70108414   7.82850409
    1.19751394]
 [ 36.13212204  28.21927071  29.89253616 ...,   6.23269033   8.46032333
    6.82706976]]


In [71]:
y = y / 96

In [72]:
print 'Y shape', y.shape

Y shape (2140, 30)


In [83]:
df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' ') )
X = np.vstack (df['Image'].values) 

In [84]:
X = X.reshape(-1,96,96)   #(7049,96,96) 图像数据

In [86]:
# Histogram equalization
for i in range(len(X)):
       X[i, :, :] = image_histogram_equalization(X[i, :,:])[0]

In [87]:
X = X.astype(np.float32)
X = X/255      #归一化
X = X.reshape(-1,1,96,96)     #（7049，1，96，96）

In [89]:
print 'Shape', 'Labels', X.shape, y.shape

Shape Labels (7049, 1, 96, 96) (2140, 30)


In [90]:
X_train = X[:1600]     #(1600, 1, 96, 96)
y_train = y[:1600]     #(1600, 30)
X_test = X[1600:]      #(5449, 1, 96, 96)
y_test = y[1600:]      #(540, 30)

In [92]:
print 'Train, Test shapes (X,y):', X_train.shape, y_train.shape, X_test.shape, y_test.shape

Train, Test shapes (X,y): (1600, 1, 96, 96) (1600, 30) (5449, 1, 96, 96) (540, 30)


In [93]:
# Train data
f = h5py.File("facialkp-train.hd5", "w")
f.create_dataset("data", data=X_train,  compression="gzip", compression_opts=4)
f.create_dataset("label", data=y_train,  compression="gzip", compression_opts=4)
f.close()

In [94]:
#Test data
f = h5py.File("facialkp-test.hd5", "w")
f.create_dataset("data", data=X_test,  compression="gzip", compression_opts=4)
f.create_dataset("label", data=y_test,  compression="gzip", compression_opts=4)
f.close()